# Prueba de un sistema de recomendaciones
**Objetivo:** evaluar los resultados de una prueba A/B de un sistema de recomendaciones mejorado

**Hipótesis:**
Tras 14 días posteriores a la inscripción hay una mejora de por lo menos el 10% en:
* Tasa de conversión de vistas a la página de producto
* Tasa de conversión en artículos agregados al carrito
* Tasa de conversión en compras

**Pasos a seguir**
1) Explorar datos
2) EDA
3) Evaluar resultados de test A/B
4) Conclusiones

## Diccionario de datos

**ab_project_marketing_events_us.csv**

El calendario de eventos de marketing para 2020
- `name`: el nombre del evento de marketing
- `regions`: regiones donde se llevará a cabo la campaña publicitaria
- `start_dt`: fecha de inicio de la campaña
- `finish_dt`: fecha de finalización de la campaña

**final_ab_new_users_upd_us.csv**

Todos los usuarios que se registraron en la tienda en línea desde el 7 hasta el 21 de diciembre de 2020
- `user_id`: id del usuario
- `first_date`: fecha de inscripción
- `region`: región del usuario
- `device`: dispositivo utilizado para la inscripción

**final_ab_events_upd_us.csv**

Todos los eventos de los nuevos usuarios en el período comprendido entre el 7 de diciembre de 2020 y el 1 de enero de 2021
- `user_id`: id del usuario
- `event_dt`: fecha y hora del evento
- `event_name`: nombre del tipo de evento
- `details`: datos adicionales sobre el evento (por ejemplo, el pedido total en USD para los eventos `purchase`)

**final_ab_participants_upd_us.csv** 

Tabla con los datos de los participantes de la prueba
- `user_id`: id del usuario
- `ab_test`: nombre de la prueba
- `group`: el grupo de prueba al que pertenecía el usuario

In [1]:
# importar librerías
import pandas as pd

# Explorar datos

In [2]:
# importar datasets
marketing_events = pd.read_csv('ab_project_marketing_events_us.csv')
new_users = pd.read_csv('final_ab_new_users_upd_us.csv')
events = pd.read_csv('final_ab_events_upd_us.csv')
participants = pd.read_csv('final_ab_participants_upd_us.csv')

In [3]:
marketing_events.info()
print('\n','Hay', marketing_events.duplicated().sum(), 'valores duplicados')
print('\n',marketing_events.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes

 Hay 0 valores duplicados

                                name                   regions    start_dt  \
4                 4th of July Promo                 N.America  2020-07-04   
5         Black Friday Ads Campaign  EU, CIS, APAC, N.America  2020-11-26   
12          Single's Day Gift Promo                      APAC  2020-11-11   
10        CIS New Year Gift Lottery                       CIS  2020-12-30   
9   Victory Day CIS (May 9th) Event                       CIS  2020-05-09   

     finish_dt  
4   2020-07-11  
5   2020-12-01  
12  2020-11-12  
10  2021-01-07  
9   2020-05-11  


In [4]:
new_users.info()
print('\n','Hay', new_users.duplicated().sum(), 'valores duplicados')
print('\n',new_users.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB

 Hay 0 valores duplicados

                 user_id  first_date     region   device
29875  876D19994C23F196  2020-12-16         EU   iPhone
25607  3165C30B7A37907F  2020-12-22         EU  Android
27888  0EDBE147491A6BB9  2020-12-16        CIS  Android
56526  4E779B0A65367C57  2020-12-20  N.America  Android
7661   701FDBA0BBD3E97D  2020-12-14       APAC   iPhone


In [5]:
events.info()
print('\n','Hay', events.duplicated().sum(), 'valores duplicados')
print('\n',events.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB

 Hay 0 valores duplicados

                  user_id             event_dt    event_name  details
384679  7D4A6CD62131BFD2  2020-12-23 17:15:21         login      NaN
187107  8572999B608671BF  2020-12-19 02:40:31  product_page      NaN
116564  CF2EE5B31800C1E7  2020-12-27 17:37:59  product_cart      NaN
170106  F7E146F475D9B81E  2020-12-17 19:45:05  product_page      NaN
135045  657BF631ABD67DCB  2020-12-11 00:58:38  product_page      NaN


In [6]:
participants.info()
print('\n','Hay', participants.duplicated().sum(), 'valores duplicados')
print('\n',participants.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB

 Hay 0 valores duplicados

                 user_id group                  ab_test
9525   F69BC50D9CC2A47B     B        interface_eu_test
11681  14E0233F5016718D     B        interface_eu_test
216    82952C2B5CFAD42B     B  recommender_system_test
3234   9F0E22D845D144F7     B  recommender_system_test
3043   146880B18CE204A9     A  recommender_system_test


### Comentario
En los DataFrames "marketing_events", "new_users" y "events" hay necesidad de realizar un procesamiento de datos.

`marketing_events` 
* start_dt y finish_dt --> convertir a formato fecha

`new_users`
* first_date --> convertir a formato fecha

`events`
* details --> estudiar valores nulos
* event_dt --> convertir a formato fecha y hora

In [7]:
# procesamiento de datos
# convertir a formato fecha
marketing_events['start_dt'] = pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt'] = pd.to_datetime(marketing_events['finish_dt'])
new_users['first_date'] = pd.to_datetime(new_users['first_date'])
events['event_dt'] = pd.to_datetime(events['event_dt'])

In [8]:
# revisar nulos en events
print(events[~events['details'].isna()].sample(10))

# rellenar nulos con código "999"
events['details'] = events['details'].fillna(999)

                user_id            event_dt event_name  details
52458  28C8E8DF93275C84 2020-12-24 22:28:08   purchase     4.99
504    2CEBE2104670753B 2020-12-07 20:02:54   purchase     4.99
38238  0D5D87FE6E78173B 2020-12-21 04:20:33   purchase     4.99
57941  C38012170A89B019 2020-12-28 20:22:53   purchase     4.99
55486  BDE51B2E33FB7014 2020-12-26 12:09:00   purchase     4.99
30610  9EB7263DA9458BA3 2020-12-18 13:05:50   purchase     9.99
54086  BE8B208A8914D6EA 2020-12-26 20:09:29   purchase     4.99
12402  EE2D392A7667C73C 2020-12-13 20:53:53   purchase     4.99
13838  9316B239FAB5707A 2020-12-13 01:27:17   purchase     4.99
5828   B3A737A6E015478B 2020-12-10 13:47:14   purchase     4.99


### Comentario
Se asignó el formato fecha a las columnas correspondientes. Los valores nulos se debían a que solo el evento "purchase" tenía los detalles de la venta (pedido en USD), sin embargo, para no dejar valores nulos se rellenaron con <span style="color: skyblue;">**el código "999" indicando que ese evento no contiene valores en "details"**</span>, esto debe de considerarse para no afectar los análisis.